In [1]:
1+1


2

In [1]:
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
import os
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [3]:
llm = OpenAI(openai_api_key= openai_api_key,temperature=0.5)

/Users/amallmuhammed/Documents/Bro Project/Intelligent-HR/project/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [5]:
s = 'Which is the biggest country in the world'
print(llm.invoke(s))



The biggest country in the world by land area is Russia, with a total area of 17,098,242 square kilometers. However, if we consider total area (including land and water), then Canada is the largest country with a total area of 9,984,670 square kilometers.


In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [7]:

job_template =  PromptTemplate(input_variables=['Job_Role'], 
                               template='What are the basic qualification required for {Job_Role}')
job_chain = LLMChain(llm=llm, prompt=job_template, output_key='Skills')

In [8]:
skill_template = PromptTemplate(input_variables=['Skills'],
                                template='The skills required to become {Job_Role} are: {Skills}')
skill_chain = LLMChain(llm=llm ,prompt=skill_template)

In [9]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[job_chain,skill_chain],
                        input_variables=['Job_Role'],
                        output_variables=['Skills'])

In [11]:
chain.invoke(Job_Role='Data Scientist')

TypeError: Chain.invoke() missing 1 required positional argument: 'input'

In [33]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [34]:
from langchain.chat_models import ChatOpenAI


QA Chat using Open AI llm
/

In [4]:
# import openai
# import re

# # Set up your OpenAI API key
# openai.api_key = "YOUR_OPENAI_API_KEY"

# def process_job_description(text):
#     """Simplifies the job description for easier question generation."""

#     # Basic processing (replace with more sophisticated techniques if needed)
#     processed_text = text.lower()  # Makes text lowercase
#     processed_text = re.sub(r"[^\w\s]", "", processed_text)  # Removes punctuation (adjust as needed) 
#     return processed_text

# def generate_questions(processed_description):
#     """Uses OpenAI's text generation to create questions."""

#     response = openai.Completion.create(
#         engine="text-davinci-003",  # Adjust the engine if needed
#         prompt=f"Generate a few insightful questions about the following job description: {processed_description}",
#         max_tokens=150,  # Controls the length of the response
#         n=1,
#         stop=None,
#         temperature=0.7,  # Encourages a bit of creativity
#     )
#     return response.choices[0].text.strip().split("\n")

# def ask_questions(questions):
#     """Asks the generated questions and gets user answers."""

#     for question in questions:
#         answer = input(question + "\nAnswer: ")
#         print(f"You answered: {answer}\n")  

# if __name__ == "__main__":
#     job_description = input("Enter a job description: \n")
#     processed_description = process_job_description(job_description)
#     questions = generate_questions(processed_description)
#     ask_questions(questions)


In [6]:
import os
import langchain as lc
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

def create_qa_chain(job_description):
    """Creates the QA chain using the provided job description."""

    # Initialize the LLM
    llm = OpenAI(temperature=0.7, openai_api_key=openai_api_key) 

    # Load the job description
    text_loader = TextLoader(job_description)
    documents = text_loader.load()

    # Split the text into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
    docs = text_splitter.split_documents(documents)

    # Create a FAISS vectorstore to store document embeddings
    vectorstore = FAISS.from_documents(docs, llm)

    # The QA chain for generating questions and answers 
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm, 
        chain_type="conversational",
        retriever=vectorstore.as_retriever()
    )

    return qa_chain

# Example usage
job_description = """
        **Job Title:** Software Engineer

        **Responsibilities:**
        * Design, develop, and test software applications.
        * Collaborate with cross-functional teams.
        * Maintain and improve existing codebases.

        **Qualifications:**
        * Bachelor's degree in Computer Science or related field.
        * Proficiency in Python or Java.
        * Experience with web development frameworks.
        """

qa_chain = create_qa_chain(job_description)

# Start a conversational session
while True:
    question = input("You: ")
    response = qa_chain.run(question)
    print(f"Bot: {response}")


RuntimeError: Error loading 
        **Job Title:** Software Engineer

        **Responsibilities:**
        * Design, develop, and test software applications.
        * Collaborate with cross-functional teams.
        * Maintain and improve existing codebases.

        **Qualifications:**
        * Bachelor's degree in Computer Science or related field.
        * Proficiency in Python or Java.
        * Experience with web development frameworks.
        

In [7]:
from transformers import pipeline

def generate_questions(job_description):
    # Initialize the QA pipeline
    qa_pipeline = pipeline("question-generation", model="valhalla/t5-base-qg-hl")

    # Prepare the prompt
    prompt = f"This is a job description:\n{job_description}\n\nCan you provide more details about the job?"

    # Generate questions
    questions = qa_pipeline(prompt, max_length=50, do_sample=True, top_p=0.95, top_k=60)

    # Extract and return the generated questions
    return [question['generated_text'] for question in questions]

# Example usage
job_description = "We are looking for a software engineer with experience in Python and machine learning."
questions = generate_questions(job_description)
for question in questions:
    print(question)


/Users/amallmuhammed/Documents/Bro Project/Intelligent-HR/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


KeyError: "Unknown task question-generation, available tasks are ['audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

In [16]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

loader = TextLoader('jobdesc.txt')
documents = loader.load()

# Create a vector store index
index = VectorstoreIndexCreator().from_loaders([loader])

# Initialize the OpenAI LLM
llm = OpenAI(temperature=0.5, openai_api_key=openai_api_key)

# Create the conversational chain
qa = ConversationalRetrievalChain.from_llm(llm, index.vectorstore)

# Start the conversation
print("Welcome to the Job Description QA! You can ask questions about the job description, and I'll try my best to answer them.")
chat_history = []

while True:
    query = input("Human: ")
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result["answer"]))
    print(f"Assistant: {result['answer']}")

ValidationError: 1 validation error for ConversationalRetrievalChain
retriever
  value is not a valid dict (type=type_error.dict)